In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns

# 1. Resumen estadístico
print(data.describe())

# 2. Verificar valores nulos
print(data.isnull().sum())

# 3. Distribución de la variable objetivo
sns.countplot(x='Survived', data=data)
plt.title('Distribución de Supervivientes')
plt.show()

# 4. Relación entre variables categóricas y la supervivencia
categorical_features = ['Pclass', 'Sex', 'Embarked']
for feature in categorical_features:
    sns.countplot(x=feature, hue='Survived', data=data)
    plt.title(f'Relación entre {feature} y Supervivencia')
    plt.show()

# 5. Distribución de edades por supervivencia
sns.histplot(data, x='Age', hue='Survived', kde=True, bins=20)
plt.title('Distribución de Edades según Supervivencia')
plt.show()

# 6. Correlación entre variables numéricas
sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
plt.title('Matriz de Correlación')
plt.show()

# 7. Detección de valores atípicos en la tarifa (Fare)
sns.boxplot(x=data['Fare'])
plt.title('Distribución de la Tarifa (Fare)')
plt.show()


In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 1. Cargar los datos 
# 2. Manejo de valores faltantes
# Rellenar valores nulos en 'Age' con la mediana
data['Age'].fillna(data['Age'].median(), inplace=True)

# Rellenar valores nulos en 'Embarked' con la moda
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

# Eliminar la columna 'Cabin' porque tiene muchos valores nulos
data.drop('Cabin', axis=1, inplace=True)

# 3. Eliminar columnas innecesarias
# Eliminamos 'Name', 'Ticket' y 'PassengerId' porque no aportan 
data.drop(['Name', 'Ticket', 'PassengerId'], axis=1, inplace=True)

# 4. Transformación de variables categóricas
label_encoder = LabelEncoder()
data['Sex'] = label_encoder.fit_transform(data['Sex'])  # male=1, female=0
data['Embarked'] = label_encoder.fit_transform(data['Embarked'])

# 5. Verificar si hay valores nulos restantes
print(data.isnull().sum())

# 6. Ver los primeros datos preprocesados
print(data.head())

from sklearn.feature_selection import SelectKBest, chi2

# Dividir en características y variable objetivo
X = data.drop('Survived', axis=1)  # Todas las características excepto la variable objetivo
y = data['Survived']               # Variable objetivo

# Aplicar SelectKBest para seleccionar las 5 características más relevantes
selector = SelectKBest(score_func=chi2, k=5)
X_new = selector.fit_transform(X, y)

# Mostrar las puntuaciones de cada característica
feature_scores = pd.DataFrame({'Feature': X.columns, 'Score': selector.scores_})
feature_scores = feature_scores.sort_values(by='Score', ascending=False)
print("Puntuaciones de características:\n", feature_scores)

# Características seleccionadas
selected_features = feature_scores.head(5)['Feature'].tolist()
print("Características más relevantes seleccionadas:", selected_features)

from sklearn.model_selection import train_test_split

# Seleccionar las características y la variable objetivo
X = data.drop('Survived', axis=1)  # Todas las columnas excepto la objetivo
y = data['Survived']               # Variable objetivo

# Dividir en conjunto de entrenamiento y prueba (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrar tamaños de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]} muestras")


from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

# Entrenar el modelo configurando hiperparámetros
decision_tree = DecisionTreeClassifier(
    criterion='gini',        # Función de evaluación: 'gini' o 'entropy'
    max_depth=5,             # Profundidad máxima del árbol
    min_samples_split=10,    # Mínimo de muestras para dividir un nodo
    min_samples_leaf=5,      # Mínimo de muestras en un nodo hoja
    random_state=42          # Reproducibilidad
)

# Entrenamiento del modelo
decision_tree.fit(X_train, y_train)

# Realizar predicciones
y_pred = decision_tree.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Mostrar resultados
print(f"Precisión del modelo: {accuracy:.2f}")
print("\nReporte de clasificación:\n", report)

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Evaluar el modelo en el conjunto de prueba
y_pred = decision_tree.predict(X_test)

# Métricas de desempeño
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Mostrar resultados
print(f"Precisión del modelo: {accuracy:.2f}")
print("\nReporte de clasificación:\n", report)

# Matriz de confusión
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Sobrevivió', 'Sobrevivió'], yticklabels=['No Sobrevivió', 'Sobrevivió'])
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Matriz de Confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Visualizar la matriz de confusión
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Sobrevivió', 'Sobrevivió'], yticklabels=['No Sobrevivió', 'Sobrevivió'])
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

from sklearn.metrics import roc_curve, auc

# Obtener las probabilidades de predicción
y_prob = decision_tree.predict_proba(X_test)[:, 1]

# Calcular la curva ROC
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Graficar la curva ROC
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color='blue', label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Línea aleatoria
plt.title('Curva ROC')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.legend(loc='lower right')
plt.show()


# Importancia de las características
importances = decision_tree.feature_importances_
indices = importances.argsort()[::-1]  # Ordenar características por importancia

# Graficar las importancias
plt.figure(figsize=(10, 6))
plt.barh(X.columns[indices], importances[indices], color='green')
plt.title('Importancia de las Características')
plt.xlabel('Importancia')
plt.ylabel('Características')
plt.show()



NameError: name 'data' is not defined